## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [10]:
import glob
import numpy as np
import scipy as sp
import scipy.io
import scipy.signal as sig
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold,train_test_split 
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from tqdm import tqdm 
import warnings
warnings.filterwarnings("ignore")

In [67]:


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]



def LoadTroikaRefFile(ref_file):
    data = sp.io.loadmat(ref_file)['BPM0']
    return data



def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)




def extract_features(ppg, accx, accy, accz, fs = 125):   
    """
    Create features from the data
    
    Returns:
        PPG and ACC features
    """
    # magnitude of accelometer
    acc=np.sqrt(accx**2 + accy**2 + accz**2)
    # frequencies and coefficients for both ppg and acceleormeter magnitude 
    fft_acc=np.fft.rfft(acc,len(acc))
    freqs_acc=np.fft.rfftfreq(len(acc),1/fs)
    fft_ppg=np.fft.rfft(ppg,len(ppg))
    freqs_ppg=np.fft.rfftfreq(len(ppg),1/fs)
    #bandpass 
    fft_ppg[freqs_ppg <= 40/60] = 0.0
    fft_ppg[freqs_ppg >= 240/60] = 0.0
        
    # getting peaks from the largest frequencies 
    #1) If the ppg peaks does not match with the acc peaks, have that in your result.
    #2) If the ppg peaks match with the acc peaks, 
    #look for the second max peak of ppg.
    #3) Sometimes, the second peak of ppg also match with the acc peak, 
    #in such case have that in your result.
    max_acc_freq=np.argmax(np.abs(fft_acc))
    max_ppg_freq=np.argmax(np.abs(fft_ppg))
    acc_peak=freqs_acc[max_acc_freq]
    ppg_peak=freqs_ppg[max_ppg_freq]
    if acc_peak==ppg_peak:
        fft_ppg=np.delete(fft_ppg,max_ppg_freq)
        freqs_ppg=np.delete(freqs_ppg,max_ppg_freq)
        max_acc_freq=np.argmax(np.abs(fft_acc))
        max_ppg_freq=np.argmax(np.abs(fft_ppg))
        acc_peak=freqs_acc[max_acc_freq]
        ppg_peak=freqs_ppg[max_ppg_freq]
    return np.array([ppg_peak, acc_peak])




def ensemble_model():       
    """
    Train Regressor model
    
    Returns:
        Trained model
    """ 
    fs = 125
    win_length = 8
    win_shift = 2
    
    # Load filenames through LoadTroikaDataset
    data_fls, ref_fls = LoadTroikaDataset()
    features, labels, signals = [], [], []
    
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
        ref=LoadTroikaRefFile(ref_fl)
        features, labels, signals = [], [], []
        fs=125
        window_length=8 *fs
        window_shift=2 *fs 
        i=0
        for index in range(0,len(ppg)- window_length,window_shift):
            ppg_w=ppg[index:index+window_length]
            accx_w=accx[index:index+window_length]
            accy_w=accy[index:index+window_length]
            accz_w=accz[index:index+window_length]
            features.append(extract_features(ppg_w, accx_w, accy_w, accz_w))
            labels.append(ref[i])
            i+=1
            signals.append([ppg_w, accx_w, accy_w, accz_w])
    
    features, labels = np.array(features), np.array(labels)
    model = AdaBoostRegressor(n_estimators=100)
    model.fit(features,labels)
    return model

def RunPulseRateAlgorithm(data_fl, ref_fl):
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    ref=LoadTroikaRefFile(ref_fl)
    print(f" len of ref ={len(ref)}")
    features, labels, signals = [], [], []
    # Load ground truth heart rate
    fs=125
    window_length=8*fs
    window_shift=2 *fs
    i=0
    for index in range(0,len(ppg)- window_length,window_shift):
        ppg_w=ppg[index:index+window_length]
        accx_w=accx[index:index+window_length]
        accy_w=accy[index:index+window_length]
        accz_w=accz[index:index+window_length]
        features.append(extract_features(ppg_w, accx_w, accy_w, accz_w))
        
        labels.append(ref[i])
        i=+1
        signals.append([ppg_w, accx_w, accy_w, accz_w])
    
    features, labels = np.array(features), np.array(labels)
    print(f" len of labels {len(labels)}")
    model = ensemble_model()
    
    # Compute pulse rate estimates and estimation confidence.
    errors, confidence = [], []
    
    for i in range(len(signals)):
        feature, label = features[i], labels[i]
        ppg, accx, accy, accz = signals[i]
        pred = model.predict(np.reshape(feature, (1, -1)))[0]      
        
        freqs = np.fft.rfftfreq(2 * len(ppg), 1 / fs)
        fft = np.abs(np.fft.rfft(ppg, 2 * len(ppg)))
        fft[freqs <= 40/60] = 0.0
        fft[freqs >= 240/60] = 0.0
    
        pred_fs = pred / 55
        pred_fs_win = (freqs >= pred_fs - 0.5) & (freqs <= pred_fs + 0.5)
        confid = np.sum(fft[pred_fs_win]) / np.sum(fft)
        
        errors.append(np.abs(pred - label))
        confidence.append(confid)
    print(f" len of errors ={len(errors)} \n ")

    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
    return errors, confidence

In [68]:
Evaluate()

 len of ref =148
 len of labels 148
 len of errors =148 
 
 len of ref =148
 len of labels 148
 len of errors =148 
 
 len of ref =140
 len of labels 140
 len of errors =140 
 
 len of ref =107
 len of labels 107
 len of errors =107 
 
 len of ref =146
 len of labels 145
 len of errors =145 
 
 len of ref =146
 len of labels 146
 len of errors =146 
 
 len of ref =150
 len of labels 150
 len of errors =150 
 
 len of ref =143
 len of labels 143
 len of errors =143 
 
 len of ref =160
 len of labels 160
 len of errors =160 
 
 len of ref =149
 len of labels 149
 len of errors =149 
 
 len of ref =143
 len of labels 142
 len of errors =142 
 
 len of ref =146
 len of labels 146
 len of errors =146 
 


ValueError: all the input array dimensions except for the concatenation axis must match exactly

-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** - Include details so someone unfamiliar with your project will know how to run your code and use your algorithm. 
> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
> - **Algorithhm Description** will include the following:
>   - how the algorithm works
>   - the specific aspects of the physiology that it takes advantage of
>   - a describtion of the algorithm outputs
>   - caveats on algorithm outputs 
>   - common failure modes
> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.

Your write-up goes here...

# Imports

In [1]:
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
%matplotlib inline 

# Exploring Data, Preprocessing and Computing Features 

-----
### Next Steps
You will now go to **Test Your Algorithm** to apply a unit test to confirm that your algorithm met the success criteria. 